#Introduction

Tire selection plays a key role in ensuring road safety, especially in extreme weather conditions. The type of tires used on your vehicle depends on the season and weather conditions. Summer tires offer excellent traction in hot, dry conditions, while winter tires offer better traction in cold, ice, and snow. All-season tires are designed to perform well in both summer and winter. 
 
The aim of this project is to develop a model for identifying the tire type, which distinguishes between summer, winter and all-season tires in particular. The data set used to train and test the model will contain images of tires with different tire wear patterns. 
 
 The development of an accurate tire type recognition model could make a significant contribution to road safety by providing drivers with an automated means of ensuring they are using the correct tires for  current weather conditions. This project aims to harness the power of deep learning to create an efficient and effective tire type recognition system.

## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Opony_Computer_Vision

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Opony_Computer_Vision


In [3]:
import requests
from bs4 import BeautifulSoup
import time
import  random
import os
from io import BytesIO
from PIL import Image
import cv2
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers

## Scraping test

In [ ]:
summer = '1'
winter = '2'
both = '3'
page = '1'


url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={summer}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page}&idmodel=0'

content = requests.get(url).content
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
images = ['https://www.24opony.pl' + image.find('img')['src'] for image in soup.find_all('a', {'rel':'prettyPhoto'})]

In [ ]:
images[0].split('/')[-1].split('.')[0]

'n_4164_s'

In [ ]:
images

In [ ]:
content = requests.get(images[0]).content

In [ ]:
img  = Image.open(BytesIO(content))

In [ ]:
img.max()

## Scraping

In [ ]:
def get_page(season, page_number):
  url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={season}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page_number}&idmodel=0'
  try:
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    time.sleep(0.5)
    return content
  except Exception as e:
    print(f'Error: {e}')

def get_urls(content):
  soup = BeautifulSoup(content, 'html.parser')
  return set(['https://www.24opony.pl' + image.find('img')['src'] for image in soup.find_all('a', {'rel':'prettyPhoto'}) if not image.find('img')['src'].split('/')[-1].split('.')[0].startswith('brak_zdjecia')])


def sharpen_image(func):
  def wrapper(url):
    img = func(url)
    new_img = cv2.filter2D(img, -1, kernel=np.array([[0, -1, 0],
                                                [-1, 5, -1],
                                                [0, -1, 0]]))
    return new_img
  return wrapper
    
@sharpen_image
def get_image(url):
  try:
    content = requests.get(url).content
    img = Image.open(BytesIO(content))
    img = np.array(img)
    return img
  except Exception as e:
    print(f'Error: {e}')

def save_image(img, url,season, path):
  file_name = url.split('/')[-1].split('.')[0] + '.jpg'
  os.makedirs(f'{path}/{season}', exist_ok=True)
  cv2.imwrite(f'{path}/{season}/{file_name}', img)

def get_data(season, start_page, end_page, path):
  for page_number in range(start_page, end_page):
    content = get_page(season, page_number + 1)
    urls = get_urls(content)

    for i, url in enumerate(urls):
      img = get_image(url)      
      save_image(img, url, season, path)
      time.sleep(1.0/100)
      if  i % 5 == 0:
        print(f'Download {i} images from page {page_number}/{end_page}')



In [ ]:
#dodaj lsitę z linkami, tak że gdy się link powtarza to skipuje

get_data(1, 1, 400, 'data/train')
get_data(2, 1, 400, 'data/train')
get_data(3, 1, 400, 'data/train')

In [ ]:
def get_page(season, page_number):
  url = f'https://www.24opony.pl/szukaj.html?f=1&idproducent=0&idsezon={season}&idrozmiar=0&idpojazd=1&czyxl=0&czyrf=0&inpre=0&cod=0&cdo=0&strona={page_number}&idmodel=0'
  try:
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    time.sleep(0.5)
    return content
  except Exception as e:
    print(f'Error: {e}')

def get_urls(content):
  soup = BeautifulSoup(content, 'html.parser')
  return set(['https://www.24opony.pl' + image['href'] for image in soup.find_all('a', {'rel':'prettyPhoto'}) if not image['href'].split('/')[-1].split('.')[0].startswith('brak_zdjecia')])


def sharpen_image(func):
  def wrapper(url):
    img = func(url)
    new_img = cv2.filter2D(img, -1, kernel=np.array([[0, -1, 0],
                                                [-1, 5, -1],
                                                [0, -1, 0]]))
    return new_img
  return wrapper
    
@sharpen_image
def get_image(url):
  try:
    content = requests.get(url).content
    img = Image.open(BytesIO(content))
    img = np.array(img)
    return img
  except Exception as e:
    print(f'Error: {e}')

def save_image(img, url,season, path):
  file_name = url.split('/')[-1].split('.')[0] + '.jpg'
  os.makedirs(f'{path}/{season}', exist_ok=True)
  cv2.imwrite(f'{path}/{season}/{file_name}', img)

def get_data(season, start_page, end_page, path):
  for page_number in range(start_page, end_page):
    content = get_page(season, page_number + 1)
    urls = get_urls(content)

    for i, url in enumerate(urls):
      img = get_image(url)      
      save_image(img, url, season, path)
      time.sleep(1.0/100)
      if  i % 5 == 0:
        print(f'Download {i} images from page {page_number}/{end_page}')


In [ ]:
# get_data(1, 1, 400, 'data/train_various_sizes')
# get_data(2, 1, 400, 'data/train_various_sizes')
get_data(3, 1, 400, 'data/train_various_sizes')

#Model

603
150


## First model

The model is based on thumbnail photos in which the tread is barely visible. The images have been sharpened.

In [ ]:
test_size = 0.2
path = 'data'

# for sub_type in ['1', '2', '3']:
#   os.makedirs(path + '/test' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train/' + sub_type + '/' + file_name,
#               path +'/test/' + sub_type + '/' + file_name)

# train_path = 'data/train'
# test_path = 'data/test'
# print(len(os.listdir(train_path + '/1')))
# print(len(os.listdir(test_path + '/1')))

In [21]:
data_path = 'data/train'

batch_size = 128
img_heights = 128
img_width = 128



In [22]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 1146 files belonging to 3 classes.
Using 917 files for training.
Found 1146 files belonging to 3 classes.
Using 229 files for validation.


In [7]:
for img, label in train_ds:
  print(img.shape)
  print(label.shape)

(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(128, 128, 128, 3)
(128, 3)
(21, 128, 128, 3)
(21, 3)


In [23]:
model = Sequential()

model.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model.add(layers.MaxPool2D(2,2))

model.add(layers.Flatten())

model.add(layers.Dense(128,
                       activation='relu'))

model.add(layers.Dense(3,
                       activation='softmax'))

In [25]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
8/8 [==============================] - 28s 3s/step - loss: 2.3039 - accuracy: 0.4482 - val_loss: 1.0842 - val_accuracy: 0.4716
Epoch 2/10
8/8 [==============================] - 25s 3s/step - loss: 1.0683 - accuracy: 0.5398 - val_loss: 1.0929 - val_accuracy: 0.4716
Epoch 3/10
8/8 [==============================] - 26s 3s/step - loss: 1.0884 - accuracy: 0.5398 - val_loss: 1.0906 - val_accuracy: 0.4716
Epoch 4/10
8/8 [==============================] - 25s 3s/step - loss: 1.0824 - accuracy: 0.5398 - val_loss: 1.0849 - val_accuracy: 0.4716
Epoch 5/10
8/8 [==============================] - 24s 3s/step - loss: 1.0716 - accuracy: 0.5398 - val_loss: 1.0771 - val_accuracy: 0.4716
Epoch 6/10
8/8 [==============================] - 25s 3s/step - loss: 1.0516 - accuracy: 0.5398 - val_loss: 1.0518 - val_accuracy: 0.4716
Epoch 7/10
8/8 [==============================] - 24s 3s/step - loss: 0.9845 - accuracy: 0.5300 - val_loss: 1.0352 - val_accuracy: 0.4760
Epoch 8/10
8/8 [==================

## Second model

A model to evaluate the impact of image quality on model efficiency. Images resolution and dimensions are higher.

### First set

The same conditions as for the previous model. An improvement in the model's efficiency can be seen, which means that the quality of the photo, and in particular the tread outlines clearly, affect the detection of the right type of tires.

In [15]:
# test_size = 0.2
# path = 'data'

# for sub_type in ['1', '2', '3']:
#   os.makedirs(path + '/test_various_sizes' +f'/{sub_type}', exist_ok=True)
#   file_list = os.listdir(path + '/train_various_sizes/' + sub_type)
#   number_of_samples = int(len(file_list) * test_size)

#   for file_name in random.sample(file_list, number_of_samples):
#     os.rename(path +'/train_various_sizes/' + sub_type + '/' + file_name,
#               path +'/test_various_sizes/' + sub_type + '/' + file_name)

# train_path = 'data/train_various_sizes'
# test_path = 'data/test_various_sizes'
# print(len(os.listdir(train_path + '/1')))
# print(len(os.listdir(test_path + '/1')))

603
150


In [27]:
data_path = 'data/train_various_sizes'

batch_size = 128
img_heights = 128
img_width = 128



In [28]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 1145 files belonging to 3 classes.
Using 916 files for training.
Found 1145 files belonging to 3 classes.
Using 229 files for validation.


In [29]:
model_1 = Sequential()

model_1.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_1.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_1.add(layers.MaxPool2D(2,2))

model_1.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_1.add(layers.MaxPool2D(2,2))

model_1.add(layers.Flatten())

model_1.add(layers.Dense(128,
                       activation='relu'))

model_1.add(layers.Dense(3,
                       activation='softmax'))

In [30]:
model_1.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
history_1 = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    batch_size=128)

Epoch 1/10
8/8 [==============================] - 30s 3s/step - loss: 0.7607 - accuracy: 0.6758 - val_loss: 0.7600 - val_accuracy: 0.6812
Epoch 2/10
8/8 [==============================] - 31s 3s/step - loss: 0.6647 - accuracy: 0.7456 - val_loss: 0.7438 - val_accuracy: 0.6769
Epoch 3/10
8/8 [==============================] - 27s 3s/step - loss: 0.6067 - accuracy: 0.7511 - val_loss: 0.7778 - val_accuracy: 0.6900
Epoch 4/10
8/8 [==============================] - 27s 3s/step - loss: 0.6786 - accuracy: 0.6921 - val_loss: 0.8494 - val_accuracy: 0.6245
Epoch 5/10
8/8 [==============================] - 29s 3s/step - loss: 0.5381 - accuracy: 0.7828 - val_loss: 0.7578 - val_accuracy: 0.7205
Epoch 6/10
8/8 [==============================] - 27s 3s/step - loss: 0.4313 - accuracy: 0.8384 - val_loss: 0.8071 - val_accuracy: 0.6638
Epoch 7/10
8/8 [==============================] - 26s 3s/step - loss: 0.3790 - accuracy: 0.8603 - val_loss: 0.7557 - val_accuracy: 0.6856
Epoch 8/10
8/8 [==================

### Second set

The number of epochs was selected as the first hyperparameter. However, the result on the test set indicates an accuracy of the magnitude of 100%, while on the validation set it is about 65%. This means that the model tends to overfitting very strongly. So the number of observations should be increased.

In [32]:
data_path = 'data/train_various_sizes'

batch_size = 128
img_heights = 128
img_width = 128



In [33]:
train_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='training',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                    interpolation='bilinear',
                                                    seed=42)

val_ds = keras.utils.image_dataset_from_directory(data_path,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    image_size = (img_heights, img_width),
                                                    batch_size = batch_size,
                                                    label_mode='categorical',
                                                  interpolation='bilinear',
                                                  seed=42)

Found 1145 files belonging to 3 classes.
Using 916 files for training.
Found 1145 files belonging to 3 classes.
Using 229 files for validation.


In [34]:
model_2 = Sequential()

model_2.add(layers.Rescaling(1/255)) # przez to, z wczytujemy batchami trzeba ustandaryzować używajać warstwy Rescaling
model_2.add(layers.Conv2D(32,
                        (3, 3),
                        padding='same',
                        activation='relu',
                        input_shape=(img_width, img_heights, 3)))

model_2.add(layers.MaxPool2D(2,2))

model_2.add(layers.Conv2D(16,
                        (3, 3),
                        padding='same',
                        activation='relu'
                        ))

model_2.add(layers.MaxPool2D(2,2))

model_2.add(layers.Flatten())

model_2.add(layers.Dense(128,
                       activation='relu'))

model_2.add(layers.Dense(3,
                       activation='softmax'))

In [35]:
model_2.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
history_2 = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
8/8 [==============================] - 28s 3s/step - loss: 0.2443 - accuracy: 0.9148 - val_loss: 0.7655 - val_accuracy: 0.6856
Epoch 2/20
8/8 [==============================] - 32s 4s/step - loss: 0.1823 - accuracy: 0.9443 - val_loss: 0.8223 - val_accuracy: 0.6812
Epoch 3/20
8/8 [==============================] - 26s 3s/step - loss: 0.1476 - accuracy: 0.9672 - val_loss: 0.8959 - val_accuracy: 0.6812
Epoch 4/20
8/8 [==============================] - 26s 3s/step - loss: 0.1145 - accuracy: 0.9738 - val_loss: 0.9259 - val_accuracy: 0.6725
Epoch 5/20
8/8 [==============================] - 29s 3s/step - loss: 0.0924 - accuracy: 0.9858 - val_loss: 0.9389 - val_accuracy: 0.6681
Epoch 6/20
8/8 [==============================] - 28s 3s/step - loss: 0.0869 - accuracy: 0.9847 - val_loss: 0.9922 - val_accuracy: 0.6550
Epoch 7/20
8/8 [==============================] - 27s 3s/step - loss: 0.0661 - accuracy: 0.9913 - val_loss: 0.9978 - val_accuracy: 0.6419
Epoch 8/20
8/8 [==================